In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

class FirstMovieSpider(scrapy.Spider):
    # Name of your spider
    name = "imdb"

    infos_movie = {}

    # Url to start your spider from
    start_urls = [
        'https://www.imdb.com/chart/top'
    ]

    def parse(self, response):

        # INFOS PRISES SUR LA PAGE DU TOP 250 

        url_film = response.css('a.ipc-title-link-wrapper').attrib['href']
        titre = response.css('ul h3.ipc-title__text::text').get()
        annee = response.css('li.ipc-metadata-list-summary-item:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > span:nth-child(1)::text').get()
        infos = response.css('span.cli-title-metadata-item::text').getall()
        note  = response.css('span.ipc-rating-star--rating::text').get()
        votes = response.css('span.ipc-rating-star--voteCount').get()
                
        self.infos_movie = {            
            'classement' : titre.split(".")[0],
            'title': titre.split(".")[1].strip(),
            'annee': annee,
            'url' : "http://imdb.com"+url_film.split("?")[0],
            'annee': infos[0],
            'duree': infos[1],
            'avertissements':infos[2],
            'note': note,
            'votes': votes.split('<!-- -->')[1]
         }
        
        # UNE FOIS TERMINE, ON VA SUR LA PAGE DU FILM
        yield response.follow(url_film, callback=self.detail_film)

    def detail_film(self, response):

        budget  = response.css('li.sc-1bec5ca1-2:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > span:nth-child(1)::text').get()
        plot = response.css('span.sc-3ac15c8d-2::text').get()
        recettes = response.css('li.sc-1bec5ca1-2:nth-child(4) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > span:nth-child(1)::text').get()

        crew = response.css('li.ipc-metadata-list__item')
        directors = crew[0].css('a.ipc-metadata-list-item__list-content-item::text').getall()
        writers = crew[1].css('a.ipc-metadata-list-item__list-content-item::text').getall()
        actors= crew[2].css('a.ipc-metadata-list-item__list-content-item::text').getall()
#section.ipc-page-section:nth-child(46) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(2) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)
        pays = response.css('section > div.dQVJPm > ul > li  > div:nth-child(2) > ul > li > a::text').getall()
        print(pays)
        self.infos_movie['plot'] = plot
        self.infos_movie['budget'] = budget
        self.infos_movie['recettes'] = recettes
        self.infos_movie['directors'] = directors
        self.infos_movie['writers'] = writers
        self.infos_movie['actors'] = actors
        #self.infos_movie['pays'] = pays

        return self.infos_movie

filename = 'infos_completes_un_film.json'

if filename in os.listdir("json/"):
    os.remove('json/'+filename)

process = CrawlerProcess(settings={
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    'FEEDS':{
        'json/' + filename : {"format": "json"},
        },
    "AUTOTHROTTLE_ENABLED": True
    }
)

#Lancer le crawler utilisant la spider
process.crawl(FirstMovieSpider)
process.start()

2024-11-13 16:54:12 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-11-13 16:54:12 [scrapy.utils.log] INFO: Versions: lxml 5.2.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.0.15 3 Sep 2024), cryptography 42.0.5, Platform Windows-11-10.0.22631-SP0
2024-11-13 16:54:12 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-13 16:54:12 [py.warnings] WARNING: c:\Users\Admin\anaconda3\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docu

['October 14, 1994 (United States)', 'United States', 'Official Facebook', 'Warner Bros. (United States)', 'English', 'Mansfield Reformatory - 100 Reformatory Road, Mansfield, Ohio, USA', 'Castle Rock Entertainment', 'Dolby Digital']
